In [61]:
import torch
import torch.nn as nn
import cv2
import numpy as np

In [62]:

import car_model


In [63]:
#创建地图
car = car_model.KinematicModel()
img = np.ones((1000,1000,3))
car.init_state((50, 50, 0))
# car.update()
# img = car.render(img)

import random

#随机生成18个障碍物
get_obstacle_pos = [(random.randint(100, 900), random.randint(100, 900)) for _ in range(18)]  # 障碍物的位置


def map_data(input,obstacle):
    img = input

    obstacle_pos = obstacle
    obstacle_size = 50
    for pos in obstacle_pos:
        top_left = (pos[0] - obstacle_size // 2, pos[1] - obstacle_size // 2)
        bottom_right = (pos[0] + obstacle_size // 2, pos[1] + obstacle_size // 2)
        cv2.rectangle(img, top_left, bottom_right, (0, 0, 0), -1)  # 用黑色表示障碍物
    #生成终点,终点是红框
    goal_pos = ((950,950))
    goal_size = 50
    top_left = (goal_pos[0] - goal_size // 2, goal_pos[1] - goal_size // 2)
    bottom_right = (goal_pos[0] + goal_size // 2, goal_pos[1] + goal_size // 2)
    cv2.rectangle(img, top_left, bottom_right, (0, 0, 255), 2)  # 用红色表示终点

    return img



In [64]:
class LidarModel:   #雷达模型
    def __init__(self,
            img_map,
            sensor_size = 21,
            start_angle = -120.0,
            end_angle = 120.0,
            max_dist = 200.0):
        self.sensor_size = sensor_size  #传感器大小
        self.start_angle = start_angle  #开始角度
        self.end_angle = end_angle  #结束角度
        self.max_dist = max_dist    #最大距离
        self.img_map = img_map  #图像地图
    
    def measure(self, pos): #测量
        sense_data = []
        inter = (self.end_angle-self.start_angle) / (self.sensor_size-1)    #间隔
        for i in range(self.sensor_size):
            theta = pos[2] + self.start_angle + i*inter
            sense_data.append(self._ray_cast(np.array((pos[0], pos[1])), theta))
        # print(sense_data)
        return sense_data


    def _ray_cast(self, pos, theta):  #射线
        x = pos[0]
        y = pos[1]
        while np.sqrt((x-pos[0])**2 + (y-pos[1])**2) < self.max_dist:
            if x < 0 or y < 0 or x >= self.img_map.shape[1] or y >= self.img_map.shape[0]:
                return self.max_dist
            
            if self.img_map[int(y), int(x)].all() == 0:
                return np.sqrt((x-pos[0])**2 + (y-pos[1])**2)
            x += np.cos(np.deg2rad(theta))  
            y += np.sin(np.deg2rad(theta))
        return self.max_dist
    
    # def render(self, pos):  #渲染
    #     img = self.img_map.copy()
    #     sense_data = self.measure(pos)
    #     # print(sense_data)
    #     for i, dist in enumerate(sense_data):
    #         theta = pos[2] + self.start_angle + i*(self.end_angle-self.start_angle) / (self.sensor_size-1)
    #         x = pos[0] + dist*np.cos(np.deg2rad(theta))
    #         y = pos[1] + dist*np.sin(np.deg2rad(theta))
    #         cv2.line(img, (int(pos[0]), int(pos[1])), (int(x), int(y)), (255, 0, 0), 1)
    #     return img

lidar = LidarModel(img)
# while True:
#     print(
#         "\rx={}, y={}, v={}, yaw={}, w={}".format(str(car.x)[:5], str(car.y)[:5], str(car.v)[:5], str(car.yaw)[:5],
#                                                   str(car.w)[:5]), end="\t")
#     car.update()
#     img = map_data(img,get_obstacle_pos)
#     img = lidar.render((car.x, car.y, car.yaw))
#     img = car.render(img)
#     img = cv2.flip(img, 0)
#     cv2.imshow("demo", img)
#     k = cv2.waitKey(1)
#     if k == ord("a"):
#         car.w += 5
#     elif k == ord("d"):
#         car.w -= 5
#     elif k == ord("w"):
#         car.v += 4
#     elif k == ord("s"):
#         car.v -= 4
#     elif k == ord("q"):
#         print()
#         break


In [65]:
#使用雷达数据，选择雷达数据中到达终点最小距离的参数，作为目标点，

class target_point:
    def __init__(self,img,lidar):
        self.lidar = lidar
        self.img = img
        self.dist = []

    def distance(self,pos1,pos2):
        return np.sqrt((pos1[0]-pos2[0])**2 + (pos1[1]-pos2[1])**2)
    
    def messure(self,pos):
        self.dist = self.lidar.measure(pos)

    def get_target_point(self,start_pos,end_pos):
        theta = 0
        f = []
        self.messure(start_pos)
        #获取雷达射线端点到达终点最小距离的索引
        for i in range(lidar.sensor_size):
            theta = start_pos[2] + self.lidar.start_angle + i*(self.lidar.end_angle-self.lidar.start_angle) / (self.lidar.sensor_size-1)
            x = start_pos[0] + self.dist[i]*np.cos(np.deg2rad(theta))
            y = start_pos[1] + self.dist[i]*np.sin(np.deg2rad(theta))
            f.append(self.distance((x,y),end_pos))
        #返回大于30的最小值索引,防止碰撞
        return f.index(min([i for i in f if i > 30]))
    
    def render(self,pos):   #渲染,最短为紫色，其他为绿色
        img = self.img.copy()
        target = self.get_target_point(pos,(950,950))
        for i in range(lidar.sensor_size):
            theta = pos[2] + self.lidar.start_angle + i*(self.lidar.end_angle-self.lidar.start_angle) / (self.lidar.sensor_size-1)
            x = pos[0] + self.dist[i]*np.cos(np.deg2rad(theta))
            y = pos[1] + self.dist[i]*np.sin(np.deg2rad(theta))
            if i == target:
                cv2.line(img, (int(pos[0]), int(pos[1])), (int(x), int(y)), (255, 0, 255), 1)
            else:
                cv2.line(img, (int(pos[0]), int(pos[1])), (int(x), int(y)), (0, 255, 0), 1)
        return img
    
target = target_point(img,lidar)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#使用RNN模型，输入为雷达最小距离的theta和小车的方向角，输出为角度
#指定小车的厨师速度为12
car.v = 12

class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(input_size=2, hidden_size=20, num_layers=1, batch_first=True)
        self.fc = nn.Linear(20, 1)
    
    def forward(self, x,h_state):
        r_out, h_state = self.rnn(x, h_state)
        out = self.fc(r_out[:, -1, :])
        return out,h_state

h_state = None
rnn = RNN().to(device)
loss = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.01)

while True:
    print(
        "\rx={}, y={}, v={}, yaw={}, w={}".format(str(car.x)[:5], str(car.y)[:5], str(car.v)[:5], str(car.yaw)[:5],
                                                  str(car.w)[:5]), end="\t")
    car.update()
    img = map_data(img,get_obstacle_pos)
    # img = lidar.render((car.x, car.y, car.yaw))
    img = target.render((car.x, car.y, car.yaw))
    img = car.render(img)
    img = cv2.flip(img, 0)
    cv2.imshow("demo", img)
    k = cv2.waitKey(1)
    if k == ord("w"):
        car.v += 4
    elif k == ord("s"):
        car.v -= 4
    elif k == ord("q"):
        print()
        break
    else:
        f = target.get_target_point((car.x,car.y,car.yaw),(950,950))
        theta = lidar.start_angle + f*(lidar.end_angle-lidar.start_angle) / (lidar.sensor_size-1)
        x = np.array([theta,car.yaw])
        x = torch.from_numpy(x).float().unsqueeze(0).unsqueeze(0).to(device)
        out , _ = rnn(x,h_state)
        car.yaw += out.item()
        loss_func = loss(out,torch.tensor(theta).to(device))
        # print(out)
        optimizer.zero_grad()
        loss_func.backward()
        optimizer.step()
cv2.destroyAllWindows()
# while True:
#     print(
#         "\rx={}, y={}, v={}, yaw={}, w={}".format(str(car.x)[:5], str(car.y)[:5], str(car.v)[:5], str(car.yaw)[:5],
#                                                   str(car.w)[:5]), end="\t")
#     car.update()
#     img = map_data(img,get_obstacle_pos)
#     # img = lidar.render((car.x, car.y, car.yaw))
#     img = target.render((car.x, car.y, car.yaw))
#     img = car.render(img)
#     img = cv2.flip(img, 0)
#     cv2.imshow("demo", img)
#     k = cv2.waitKey(2)
#     if k == ord("a"):
#         car.w += 5
#     elif k == ord("d"):
#         car.w -= 5
#     elif k == ord("w"):
#         car.v += 4
#     elif k == ord("s"):
#         car.v -= 4
#     elif k == ord("q"):
#         print()
#         break

x=53.59, y=50.02, v=12, yaw=1.675, w=0	

/home/mrxiuer/.conda/envs/pytorch/lib/python3.9/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


x=138.0, y=140.4, v=12, yaw=49.15, w=0	

KeyboardInterrupt: 